## Applied Data Science Capstone Final Project ##


### Introduction

This assignment aim explore and cluster the neighborhoods in Toronto. 

Scraping the Wikipedia page, in order to obtain the data is in the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

### 1.0 Importing necessary libraries

In [1]:
# 1 | importing libraries:

import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import re

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [49]:
import matplotlib.cm as cm
import matplotlib.colors as colors

### 1.1 Opening html and converting to Pandas Data Frame

In [2]:
# 2 | opening data frame:

# url path
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text

# store the contents of the website under doc:
doc = BeautifulSoup(wiki_html, 'html.parser')

data = []
for tr in doc.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

### 1.2 Data preprocessing and cleaning data frame:

In [3]:
# 3 | formating data frame:

# convert in data frame: 
df = pd.DataFrame(data=data)

# concatening the columns:

df0 = pd.Series(df[0])
df1 = pd.Series(df[1])
df2 = pd.Series(df[2])
df3 = pd.Series(df[3])
df4 = pd.Series(df[4])
df5 = pd.Series(df[5])
df6 = pd.Series(df[6])
df7 = pd.Series(df[7])
df8 = pd.Series(df[8])

df = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8])

In [4]:
# convert in data frame: 
df = pd.DataFrame(data=df)

# droping not assigned:
df = teste = df[~df[0].str.contains("Not assigned")]

# reseting index:
df.reset_index(inplace = True, drop=True)

df.head()

,0
0,M1BScarborough(Malvern / Rouge)
1,M1CScarborough(Rouge Hill / Port Union / Highl...
2,M1EScarborough(Guildwood / Morningside / West ...
3,M1GScarborough(Woburn)
4,M1HScarborough(Cedarbrae)


In [5]:
# create blank df with the columns PostaCode, Borough and Neighrborhood:

cols = ['df_raw','PostalCode', 'Borough', 'Neighborhood']
df_zeros = pd.DataFrame(0, index=np.arange(len(df)), columns=cols)

print(df_zeros.shape)

(103, 4)


In [6]:
# transforming in string:

df_zeros['df_raw'] = df[0].astype(str)
df_zeros.head()

,df_raw,PostalCode,Borough,Neighborhood
0,M1BScarborough(Malvern / Rouge),0,0,0
1,M1CScarborough(Rouge Hill / Port Union / Highl...,0,0,0
2,M1EScarborough(Guildwood / Morningside / West ...,0,0,0
3,M1GScarborough(Woburn),0,0,0
4,M1HScarborough(Cedarbrae),0,0,0


In [7]:
# taking the neighborhoods:

#for i in range(len(df_zeros)):
#   df_zeros["Neighborhood"].iloc[i] = df_zeros["df_raw"].iloc[i][df_zeros["df_raw"].iloc[i].find("(")+1:df_zeros["df_raw"].iloc[i].find(")")]
    
#df_zeros.head()

In [8]:
# taking the postal code:

for i in range(len(df_zeros)):
    df_zeros['PostalCode'].iloc[i] = df_zeros['df_raw'].iloc[i][0:3]
    
# taking the borough:

for i in range(len(df_zeros)):
    df_zeros['Borough'].iloc[i] = df_zeros['df_raw'].iloc[i][3:]
    
# taking the neighborhood and replacing "/" to "," :
    
for i in range(len(df_zeros)):
    df_zeros["Neighborhood"].iloc[i] = df_zeros["Borough"].iloc[i][df_zeros["Borough"].iloc[i].find("(")+1:df_zeros["Borough"].iloc[i].find(")")]
    
for i in range(len(df_zeros)):
    a = df_zeros['Borough'].iloc[i]
    b = a.split('(')
    df_zeros['Borough'].iloc[i] = b[0]
    
for i in range (len(df_zeros)):
    df_zeros["Neighborhood"].iloc[i] = df_zeros["Neighborhood"].iloc[i].replace(' /',',')

df_zeros.head()

C:\Users\Frederico\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,df_raw,PostalCode,Borough,Neighborhood
0,M1BScarborough(Malvern / Rouge),M1B,Scarborough,"Malvern, Rouge"
1,M1CScarborough(Rouge Hill / Port Union / Highl...,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1EScarborough(Guildwood / Morningside / West ...,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1GScarborough(Woburn),M1G,Scarborough,Woburn
4,M1HScarborough(Cedarbrae),M1H,Scarborough,Cedarbrae


### 1.4 Final dataframe with Postal Code, Borough and Neighborhood and export

In [9]:
# df:

df = df_zeros[['PostalCode', 'Borough','Neighborhood']]
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
# shape the final data frame

df.shape

(103, 3)

In [11]:
# Export the dataframe
df.to_csv(r'Canada_Neighborhood.csv',index=None,header=True)

### 2.0 Importing csv file cointaing the latitudes and logititudes of the neighborhoods in Canada:

In [12]:
# reading longitude/latitude table:

lonlat=pd.read_csv('Geospatial_Coordinates.csv')
lonlat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### 2.1 Prerocessing the new table and merging for getting latitudes and longitudes for various neighborhoods im Canada

In [13]:
# Change the name "Postal Code" in lonlat to "PostalCode"
lonlat.rename(columns={'Postal Code':'PostalCode'},inplace=True)

# Left join
df_geo=pd.merge(df,lonlat,how='left',on='PostalCode')

df_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### 2.2 Separating Toronto dataframe for next task

In [14]:
# separating Toronto dataframe

df_trt = df_geo[df_geo.Borough.str.contains('Toronto')]

In [15]:
#shape of Toronto Dataframe

df_trt.shape

(39, 5)

### 2.3 Visualizing all the Neighbourhoods of data frame using Folium

In [16]:
# creating Toronto map:

geolocator = Nominatim()
address = 'Toronto, Canada'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))

C:\Users\Frederico\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinates of Toronto, Canada are 43.653963, -79.387207.


### 2.4 Create map of Toronto using latitude and longitude values:

In [17]:
# create map of Toronto using latitude and longitude values
map_trt = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_trt['Latitude'], df_trt['Longitude'], df_trt['Borough'], df_trt['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='white',
        fill_opacity=0.7).add_to(map_trt)  
    
map_trt

### 3.0 Using the foursquare API to explore and cluster the neighborhoods in Toronto

In [52]:
# define Foursquare Credentials and Version:

CLIENT_ID = '...' #shade de client id
CLIENT_SECRET = '...' #shade client secret
VERSION = '20180605' # Foursquare API version
LIMIT=100 # limit of number of venues returned by Foursquare API
radius=500 # define radius

In [53]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=...&client_secret=...&v=20180605&ll=43.6627439,-79.321558&radius=500&limit=100'

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
results = requests.get(url).json()
#results

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rorschach Brewing Co.,Brewery,43.663483,-79.319824
1,Leslieville Farmers Market,Farmers Market,43.664901,-79.319784
2,The Sidekick,Comic Shop,43.664484,-79.325162
3,Queen Margherita Pizza,Pizza Place,43.664685,-79.324164
4,Chino Locos,Burrito Place,43.664653,-79.325584


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


### 5 )

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
# This will generate venues for each neighborhood 
toronto_venues = getNearbyVenues(names=df_trt['Neighborhood'],
                                   latitudes=df_trt['Latitude'],
                                   longitudes=df_trt['Longitude'])

The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

In [30]:
print("The shape of the dataframe is {}. The dataset has {} rows.".format(toronto_venues.shape,toronto_venues.shape[0]))
toronto_venues.head()

The shape of the dataframe is (1691, 7). The dataset has 1691 rows.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [31]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


### 6.2)

In [32]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,86,86,86,86,86,86
Christie,17,17,17,17,17,17
Church and Wellesley,90,90,90,90,90,90
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36
Davisville North,9,9,9,9,9,9


### 7)

### Analyze Each Neighborhood

In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood']=toronto_venues['Neighborhood']
# Note: In the lab, the column name in the dummy variale df is the same as the one in no-dv df. Here it won't work.
#       Column names must be different.

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# check the number of rows. It must be equal to that of toronto_venues
print("The shape of the dataframe is {}. The dataset has {} rows.".format
      (toronto_onehot.shape,toronto_onehot.shape[0]))

# print top 5 rows
toronto_onehot.head()

The shape of the dataframe is (1691, 238). The dataset has 1691 rows.


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [35]:
toronto_onehot.shape

(1691, 238)

#### NGrouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

# Check the shape of toronto_grouped. The number of rows is equal to that of toronto_data (both are grouped by neighborhood)
print("The shape of the dataframe is {}. The dataset has {} rows.".format
      (toronto_grouped.shape,toronto_grouped.shape[0]))

The shape of the dataframe is (39, 238). The dataset has 39 rows.


Printing each neighborhood along with the top 10  most common venues

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.0,Coffee Shop,Café,Restaurant,Cocktail Bar,Bakery,Beer Bar,Farmers Market,Cheese Shop,Seafood Restaurant,Eastern European Restaurant
1,0.0,Breakfast Spot,Nightclub,Café,Coffee Shop,Yoga Studio,Gym,Pet Store,Performing Arts Venue,Japanese Restaurant,Italian Restaurant
2,0.0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Bar,Sculpture Garden,Coffee Shop,Boat or Ferry,Harbor / Marina,Plane
3,0.0,Coffee Shop,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Burger Joint,Ice Cream Shop,Juice Bar,Department Store,Thai Restaurant
4,0.0,Grocery Store,Café,Park,Nightclub,Diner,Candy Store,Restaurant,Gas Station,Baby Store,Coffee Shop


### Clustering neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [45]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 0,
       2, 2, 2, 2, 4, 1, 2, 2, 2, 2, 2, 2, 0, 4, 2, 2, 2])

In [46]:
# add clustering labels

toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

# Left join
toronto_merged=pd.merge(df_trt,toronto_grouped,how='left',left_on='Neighborhood',right_on='Neighbourhood')

# Delete duplicate neighborhood column
toronto_merged.drop(['Neighbourhood'],axis=1,inplace=True)

# Rename column
toronto_merged.rename(columns={'Neighborhood_x':'Neighborhood'},inplace=True)

# Check top 5 rows
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,0.0,0.0,0.0,0.0,...,0.20000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.00000
1,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,4,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.00000
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,0.0,0.0,0.0,0.0,...,0.02439,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.02439
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.00000
4,M4M,East Toronto,Studio District,43.659526,-79.340923,2,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.025,0.0,0.0,0.0,0.02500


In [47]:
# Check the shape of toronto_merged
toronto_merged.shape

(39, 243)

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# Screenshot of the map created above
PATH = "D:/Learning/Applied Data Science/Course 4 Applied Data Science Capstone/Week 3/"
Image(filename = PATH + "Clustered TRT.png")